In [1]:
from functions import *
import math
import numpy as np
import torch as t
from torch import tensor
import scipy.constants as con
import matplotlib.pyplot as plt
import torch.linalg as ln
import scipy
import scipy.stats as stats
from mpl_toolkits import mplot3d
import variables as vr


In [2]:
class Transition:
    def __init__(self,DR,LL,UL,particle):
        #creating a transition requires the natural decay rate, the lower level, and the upper level, and finally the particle for which the transition applies
        #DR is the decay rate, given in Hz
        #LL is the lower level, a string in the "levels" dictionary of the particle
        #Ul is the upper level"                                                     "
        #particle must be a class with the properties assigned
        #the transition automatically calculates the M_F of the upper and lower levels based on the F in the upper and lower level designation
        self.DR=DR
        self.LL=LL
        self.UL=UL
        self.Ml=int(LL[LL.index("F")+1])
        self.Mu=int(UL[UL.index("F")+1])
        self.Energy=particle.levels[UL]-particle.levels[LL]


class species:
    def __init__(self, mass, energy_levels, isotope, nuclear_spin):
        self.mass=mass
        self.levels=energy_levels
        self.isotope=isotope
        self.nspin=nuclear_spin
        self.dtype='species_indicator'




#Rubidium Data
rubidium=species(
    con.atomic_mass*86.909180520,
    #defining energy levels should be expandable, need to dynamically assign energy levels (in terms of transitions and/or in terms of levels?)
    {
        "S12F1":0,
        "S12F2":4.5283e-24,
        "P12F0":2.54593588e-19,
        "P12F1":2.54593636e-19,
        "P12F2":2.545937398e-19,
        "P12F3":2.54593917e-19
    },
    87,
    1.5
)
    
    

#defining the transitions we care about
rubidium.D2cooling=Transition(38.11e6,"S12F2","P12F3",rubidium)
rubidium.D2repump=Transition(38.11e6,"S12F1","P12F2",rubidium)
rubidium.D2decay2=Transition(38.11e6,"S12F2","P12F2",rubidium)

In [3]:
class laserbeam:
    def __init__(self,passes_through, direction, wavelength, profile,def_device=vr.def_device):
        #passes_through gives one point where the beam passes through (default=[0,0,0])
        #direction gives the direction in which the beam moves
        #wavelength gives the wavelength of the light in the beam.
        #profile is given as a matrix of intensities with the center of the matrix being the center of the beam (expected type is np matrix)
        #profile is measured with grid squares of size .1*.1 mm
        #profile indicates the beam shape and intensities, not wavelenght profile.
        if passes_through==0:
            self.P0=tensor(np.array([0,0,0]),device=def_device,dtype=t.float64)
        self.k=2*math.pi/wavelength
        direction=tensor(direction,device=def_device,dtype=t.float64)
        self.dir=t.divide(direction,t.norm(direction,dim=0))
        self.kvect=self.dir*self.k
        self.profile=profile



In [4]:
#defining cooling lasers
Int=100
Cooling1=laserbeam(0,[1,0,0],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Cooling2=laserbeam(0,[-1,0,0],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Cooling3=laserbeam(0,[0,1,0],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Cooling4=laserbeam(0,[0,-1,0],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Cooling5=laserbeam(0,[0,0,1],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Cooling6=laserbeam(0,[0,0,-1],780e-9,cgaussianprofile(Int,0.01,0.02, vr.def_device))
Laserbeams=[Cooling1,Cooling2,Cooling3,Cooling4,Cooling5,Cooling6]
for i in Laserbeams:
    print(disttobeam( tensor([1,2,3],device=vr.def_device),i))

tensor([3.6056], dtype=torch.float64)
tensor([3.6056], dtype=torch.float64)
tensor([3.1623], dtype=torch.float64)
tensor([3.1623], dtype=torch.float64)
tensor([2.2361], dtype=torch.float64)
tensor([2.2361], dtype=torch.float64)


tensor([[ 0.0042,  0.6860,  0.6787],
        [-0.9149,  0.8598,  0.1872],
        [ 0.0938, -0.9773,  0.6527],
        ...,
        [ 0.6612,  0.8055, -0.3620],
        [-0.2284, -0.8904,  0.1760],
        [ 0.1313,  0.6208,  0.2396]], dtype=torch.float64)
[0.96501273 0.87995486 1.17521274 ... 0.88308258 0.90767383 0.66538924]


How to treat particles, should I have them be included in each specie, or should i create individual objects, with the overall class of particle. if the latter, how am I to perform the calculations needed in parallel, for this it is likely needed to have a set of arrays which store the positions, velocities, and energy level distributions of the particles. Perhaps the energy level distributions can be modeled in an ad-hoc manner for the high-velocity particles, however this method will likely not hold for lower energies (which may be fine). This is not an easy problem.


Awnser: Each particle can be classified by their position, velocity, species, and (energy level distribution).
There is no need to place the particles into their species class, nor to make a seperarte class of object called particles. Can simply work with the arrays for each themselves. Might need sets of pointers for each species. but that is trivial to generate.

In [5]:

#this system has been deprecated as unneeded.

'''
class off_limit_volumes:
    #this class is meant to define domain boundaries, i.e. boundaries for the particles.
    #I will handle this by making particles which collide with these boundaries dissapear.
    #these boundaries themselves will be defined by a tetrahedron, which has the benefit that it is fully connected,
    #allowing for it to be formed by simply defining 4 points. 
    #This allows simple tests for whether a given point lies within this tetrahedron.
    def __init__():
'''


#need to sort out how to assign `rules` modularly, for now the following rule will create a pyramid which will do.

def rule(x,def_device=vr.def_device):
    x,y,z=x[:,0],x[:,1],x[:,2]
    A=t.max(abs(x),abs(y))-z-0.1*t.ones(x.shape[0],device=def_device)
    A+=t.abs(A)
    A=t.ceil(A)
    A=(A==t.zeros(A.shape[0],device=A.device))
    return A


In [16]:

class particles:
    #the particles class contains the data for the particles included in the model, this data includes particle position, velocity, species, and 
    def create(positions,velocities,species):
        particles.x=positions
        particles.v=velocities
        #need species to be represented as a numpy array, as torch tensors cant include general pointers
        if species.dtype==object:
            particles.type=species
        else:
            particles.type=np.array([species]).repeat(positions.shape[0])

    def createbyT(N,species,T=300,R=0.01,def_device=vr.def_device):
        #produces a cloud of N particles at radius R, as though particles wander into this sphere and come into the simulated surface


        #randomly distribute points over a shell at radius R. 
        particles.x=randthreevecs(N,def_device)
        # Need to use the chi4 distribution as we're not interested in all the velocies in the gas chamber(which would be maxwell distribution (chi3)) but
        #  instead only those which pass te boundary, which is effusion, which must be multiplied by v_n, where v_n is the velocity into the  sphere. 
        # This also requires the modification of the direction distribution. But doing this correctly allows for a simplified insertion of particles to the model
        #
        # This mode of addition is neccesarily based off of the ideal gas model.
        v_therm=math.sqrt(2*con.k*T/species.mass)
        #see notes
        vel=stats.chi.rvs(4,size=N,scale=v_therm)*2/math.pi
        phi=t.zeros((N),device=def_device).uniform_(0,2*np.pi)
        theta=t.arcsin(t.sqrt(t.zeros((N),device=def_device).uniform_()))
        
        nx=(t.outer(particles.x[:,2],t.tensor([0,1,0],device=def_device))-t.outer(particles.x[:,1],t.tensor([0,0,1],device=def_device))).T
        ny=(t.outer(particles.x[:,2],t.tensor([1,0,0],device=def_device))-t.outer(particles.x[:,0],t.tensor([0,0,1],device=def_device))).T
        nx=t.divide(nx,t.norm(nx,dim=0))
        ny=t.divide(ny,t.norm(ny,dim=0))
        particles.v=(tensor(vel,device=def_device)*(t.mul(t.cos(theta),particles.x.T)+t.mul(t.sin(theta),(t.mul(t.cos(phi),nx)+t.mul(t.sin(phi),ny))))).T
        if species.dtype==object:
            particles.type=species
        else:
            particles.type=np.array([species]).repeat(N)
        particles.x=R*particles.x
        
        
    def add(positions,velocities,species):
        particles.x=t.cat((particles.x,positions))
        particles.v=t.cat((particles.v,velocities))
        #need species to be represented as a numpy array, as torch tensors cant include general pointers
        if species.dtype==object:
            particles.type=np.append(particles.type,species)
        else:
            particles.type=np.append(particles.type,np.array([species]).repeat(positions.shape[0]))
    
    def addbyT(N,species,T=300,R=0.01,def_device=vr.def_device):
        #produces a cloud of N particles at radius R, as though particles wander into this sphere and come into the simulated surface


        #randomly distribute points over a shell at radius R. 
        x=randthreevecs(N,def_device)
        # Need to use the chi4 distribution as we're not interested in all the velocies in the gas chamber(which would be maxwell distribution (chi3)) but
        #  instead only those which pass te boundary, which is effusion, which must be multiplied by v_n, where v_n is the velocity into the  sphere. 
        # This also requires the modification of the direction distribution. But doing this correctly allows for a simplified insertion of particles to the model
        #
        # This mode of addition is neccesarily based off of the ideal gas model.
        v_therm=math.sqrt(2*con.k*T/species.mass)
        #see notes
        vel=stats.chi.rvs(4,size=N,scale=v_therm)*2/math.pi
        phi=t.zeros((N),device=def_device).uniform_(0,2*np.pi)
        theta=t.arcsin(t.sqrt(t.zeros((N),device=def_device).uniform_()))
        nx=(t.outer(x[:,2],t.tensor([0,1,0],device=def_device))-t.outer(x[:,1],t.tensor([0,0,1],device=def_device))).T
        ny=(t.outer(x[:,2],t.tensor([1,0,0],device=def_device))-t.outer(x[:,0],t.tensor([0,0,1],device=def_device))).T
        nx=t.divide(nx,t.norm(nx,dim=0))
        ny=t.divide(ny,t.norm(ny,dim=0))
        v=(tensor(vel,device=def_device)*(t.mul(t.cos(theta),x.T)+t.mul(t.sin(theta),(t.mul(t.cos(phi),nx)+t.mul(t.sin(phi),ny))))).T
        if species.dtype==object:
            type=species
        else:
            type=np.array([species]).repeat(N)
        x=R*x
        particles.x=t.cat((particles.x,x))
        particles.v=t.cat((particles.v,v))
        particles.type=np.append(particles.type,type)


particles.createbyT(100,rubidium)
print(particles.x.shape)
'''
particles.createbyT(100000,rubidium)
print(particles.v)
plt.hist(particles.v.T,30)
'''

torch.Size([180, 3])


'\nparticles.createbyT(100000,rubidium)\nprint(particles.v)\nplt.hist(particles.v.T,30)\n'